## Comp2-1 量表类型数据

主要适配于临床数据的建模和刻画。典型的应用场景探究rad_score最终临床诊断的作用。

## Onekey步骤

1. 数据校验，检查数据格式是否正确。
3. 查看一些统计信息，检查数据时候存在异常点。
4. 正则化，将数据变化到服从 N~(0, 1)。
5. 通过相关系数，例如spearman、person等筛选出特征。
6. 构建训练集和测试集，这里使用的是随机划分，正常多中心验证，需要大家根据自己的场景构建两份数据。
7. 通过Lasso筛选特征，选取其中的非0项作为后续模型的特征。
8. 使用机器学习算法，例如LR、SVM、RF等进行任务学习。
9. 模型结果可视化，例如AUC、ROC曲线，混淆矩阵等。


# 筛选临床特征

需要根据自己的情况，筛选特征，一般情况筛选pvalue<0.05的。

In [ ]:
import pandas as pd
import numpy as np
from collections import namedtuple
from onekey_algo import OnekeyDS as okds
from onekey_algo import get_param_in_cwd
import onekey_algo.custom.components as okcomp
from onekey_algo.custom.components.comp1 import fillna
import matplotlib.pyplot as plt

task = 'label'
task_type = 'Clinical'
group_info = 'group'
plt.rcParams['figure.dpi'] = 300

## 一、数据校验
首先需要检查诊断数据，如果显示`检查通过！`择可以正常运行之后的，否则请根据提示调整数据。

数据文件中的数据都是数值类型，或者可以映射成数值类型，这里的`label`某些情况下可能是非数值的，需要自定义数值函数。

**注意：在使用树模型时，可以存在缺失，但是线性模型不允许缺失，请自行根据需要填充缺省值**

In [ ]:
data_file = r'data/clinic_mul.csv'
labels = [task]
featrues_not_use = ['ID']

structed_data = pd.read_csv(data_file, header=0)
structed_data

### 特征维度

In [ ]:
# 删掉ID这一列。
ids = structed_data['ID']
structed_data = structed_data.drop(featrues_not_use, axis=1)
structed_data

## 二、数据统计

1. count，统计样本个数。
2. mean、std, 对应特征的均值、方差
3. min, 25%, 50%, 75%, max，对应特征的最小值，25,50,75分位数，最大值。

In [ ]:
structed_data.describe()

## 三、正则化

临床特征可选是否正则化，默认不进行

`normalize_df` 为onekey中正则化的API，将数据变化到0均值1方差。正则化的方法为

$column = \frac{column - mean}{std}$

In [ ]:
# from onekey_algo.custom.components.comp1 import normalize_df
# data = normalize_df(structed_data, not_norm=labels + ['group'])
# data = data.dropna(axis=1)
# data.describe()
data = structed_data
sel_data = structed_data

## 四、相关系数

计算相关系数的方法有3种可供选择
1. pearson （皮尔逊相关系数）: standard correlation coefficient

2. kendall (肯德尔相关性系数) : Kendall Tau correlation coefficient

3. spearman (斯皮尔曼相关性系数): Spearman rank correlation

三种相关系数参考：https://blog.csdn.net/zmqsdu9001/article/details/82840332

In [ ]:
# 如果需要选择相关系数使用对应的相关系数即可。
# pearson_corr = data.corr('pearson')
# kendall_corr = data.corr('kendall')
spearman_corr = data.corr('spearman')

import seaborn as sns
import matplotlib.pyplot as plt
from onekey_algo.custom.components.comp1 import draw_matrix
plt.figure(figsize=(10.0, 8.0))

# 选择可视化的相关系数
draw_matrix(spearman_corr, annot=True, cmap='YlGnBu', cbar=False)
plt.savefig(f'img/Clinic_feature_corr.svg', bbox_inches = 'tight')

## 五、构建数据

将样本的训练数据X与监督信息y分离出来，并且对训练数据进行划分，一般的划分原则为80%-20%

In [ ]:
import numpy as np
import onekey_algo.custom.components as okcomp
from collections import OrderedDict
n_classes = 2

sel_data = data
train_data = sel_data[(sel_data[group_info] == 'train')]
train_ids = ids[train_data.index]
train_data = train_data.reset_index()
train_data = train_data.drop('index', axis=1)
y_data = train_data[labels]
X_data = train_data.drop(labels + [group_info], axis=1)

subsets = [s for s in get_param_in_cwd('subsets') if s != 'train']
val_datasets = OrderedDict()
for subset in subsets:
    val_data = sel_data[sel_data[group_info] == subset]
    val_ids = ids[val_data.index]
    val_data = val_data.reset_index()
    val_data = val_data.drop('index', axis=1)
    y_val_data = val_data[labels]
    X_val_data = val_data.drop(labels + [group_info], axis=1)
    val_datasets[subset] = [X_val_data, y_val_data, val_ids]

y_all_data = sel_data[labels]
X_all_data = sel_data.drop(labels + [group_info], axis=1)

column_names = X_data.columns
print(f"训练集样本数：{X_data.shape}，", '，'.join([f"{subset}样本数：{d_[0].shape}" for subset, d_ in val_datasets.items()]))

## 六、模型筛选

根据筛选出来的数据，做模型的初步选择。当前主要使用到的是Onekey中的

1. SVM，支持向量机，引用参考。
2. KNN，K紧邻，引用参考。
3. Decision Tree，决策树，引用参考。
4. Random Forests, 随机森林，引用参考。
5. XGBoost, bosting方法。引用参考。
6. LightGBM, bosting方法，引用参考。

In [ ]:
model_names = get_param_in_cwd('ml_models')
models = okcomp.comp1.create_clf_model(model_names)
model_names = list(models.keys())

### 交叉验证

`n_trails`指定随机次数，每次采用的是80%训练，随机20%进行测试，找到最好的模型，以及对应的最好的数据划分。

这里的数据并没有使用前面`Lasso`筛选出来的特征进行训练，理论来说，特征筛选仅对线性模型有一定作用，例如`SVM`、`LR`，但是对树模型没什么作用，例如`DecisionTree`、`Random`这些。所以默认不筛选。

```python
def get_bst_split(X_data: pd.DataFrame, y_data: pd.DataFrame,
            models: dict, test_size=0.2, metric_fn=accuracy_score, n_trails=10,
            cv: bool = False, shuffle: bool = False, metric_cut_off: float = None, random_state=None):
    """
    寻找数据集中最好的数据划分。
    Args:
        X_data: 训练数据
        y_data: 监督数据
        models: 模型名称，Dict类型、
        test_size: 测试集比例
        metric_fn: 评价模型好坏的函数，默认准确率，可选roc_auc_score。
        n_trails: 尝试多少次寻找最佳数据集划分。
        cv: 是否是交叉验证，默认是False，当为True时，n_trails为交叉验证的n_fold
        shuffle: 是否进行随机打乱
        metric_cut_off: 当metric_fn的值达到多少时进行截断。
        random_state: 随机种子

    Returns: {'max_idx': max_idx, "max_model": max_model, "max_metric": max_metric, "results": results}

    """
```

**注意：这里采用了【挑数据】，如果想要严谨，请修改`n_trails=1`。**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score

# 随机使用n_trails次数据划分，找到最好的一次划分方法，并且保存在results中。
results = okcomp.comp1.get_bst_split(X_data, y_data, models, test_size=0.2, metric_fn=roc_auc_score, n_trails=5, cv=True, random_state=0)
# _, (X_train_sel, X_test_sel, y_train_sel, y_test_sel) = results['results'][results['max_idx']]
trails, _ = zip(*results['results'])
cv_results = pd.DataFrame(trails, columns=model_names)
# 可视化每个模型在不同的数据划分中的效果。
sns.barplot(data=cv_results)
plt.ylabel('AUC %')
plt.xlabel('Model Nmae')
plt.xticks(rotation=90)
plt.ylim(0.5,)
plt.savefig(f'img/{task_type}_model_cv.svg', bbox_inches = 'tight')

### 模型筛选

使用最好的数据划分，进行后续的模型研究。

**注意**: 一般情况下论文使用的是随机划分的数据，但也有些论文使用【刻意】筛选的数据划分。

In [ ]:
# import joblib
# import os
# from onekey_algo.custom.components.comp1 import plot_feature_importance, smote_resample
# from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score
# from sklearn.preprocessing import OneHotEncoder
# from onekey_algo.custom.components.delong import calc_95_CI
# from onekey_algo.custom.components.metrics import analysis_pred_binary


# os.makedirs('models', exist_ok=True)
# sel_model = 'RandomForest'
# maxi = 0
# for p in range(1, 100):
#     for md in range(1, 5):
#         targets = []
#         print(p, md)
#         for l in labels:
#             new_models = okcomp.comp1.create_clf_model([sel_model])
# #             new_models['LR'] = LogisticRegression(penalty='none', max_iter=10)
#         #     new_models['SVM'] = SVC(probability=True, kernel='rbf')
#             if sel_model == 'RandomForest':
#                 new_models['RandomForest'] = RandomForestClassifier(n_estimators=p, max_depth=md, min_samples_split=2, random_state=0)
#             elif sel_model == 'ExtraTrees':
#                 new_models['ExtraTrees'] = ExtraTreesClassifier(n_estimators=p, max_depth=md, min_samples_split=2, random_state=0)
#             elif sel_model == 'LightGBM':
#                 new_models['LightGBM'] = LGBMClassifier(n_estimators=p, max_depth=md, random_state=0)
#             elif sel_model == 'XGBoost':
#                 new_models['XGBoost'] = XGBClassifier(n_estimators=p, objective='binary:logistic', max_depth=md, min_child_weight=.2,
#                                                       use_label_encoder=False, eval_metric='error')            
#             model_names = list(new_models.keys())
#             new_models = list(new_models.values())
#             for mn, m in zip(model_names, new_models):        
#                 X_train_sel, y_train_sel = X_data, y_data
# #                 X_train_sel, y_train_sel = smote_resample(X_train_sel, y_train_sel)
#                 m.fit(X_train_sel, y_train_sel[l])
#                 # 保存训练的模型
#         #         joblib.dump(m, f'models/{task_type}_{mn}_{l}.pkl') 
#                 # 输出模型特征重要性，只针对高级树模型有用
#         #         plot_feature_importance(m, selected_features[0], save_dir='img', prefix=f"{task_type}_")
#             targets.append(new_models)


#         metric = []
#         pred_sel_idx = []
#         X_train_sel, y_train_sel = X_data, y_data
#         predictions = [[(model.predict(X_train_sel), 
#                          [(model.predict(X_val_sel), y_val_sel) for X_val_sel, y_val_sel, _ in val_datasets.values()])  
#                         for model in target] for label, target in zip(labels, targets)]
#         pred_scores = [[(model.predict_proba(X_train_sel), 
#                          [(model.predict_proba(X_val_sel), y_val_sel) for X_val_sel, y_val_sel, _ in val_datasets.values()]) 
#                         for model in target] for label, target in zip(labels, targets)]

        
#         for label, prediction, scores in zip(labels, predictions, pred_scores):
#             for mname, (train_pred, val_preds), (train_score, val_scores) in zip(model_names, prediction, scores):
#                 # 计算训练集指数
#                 acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y_train_sel[label], 
#                                                                                                       train_score[:, 1])
#                 ci = f"{ci[0]:.3f} - {ci[1]:.3f}"
#                 metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, f"train"))
#                 for subset, (val_score, y_val_sel) in zip(val_datasets.keys(), val_scores):
#                     acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y_val_sel[label], 
#                                                                                                           val_score[:, 1])
#                     ci = f"{ci[0]:.3f} - {ci[1]:.3f}"
                    
                    
#                     metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, subset))
#         metric = pd.DataFrame(metric, index=None, columns=['model_name', 'Accuracy', 'AUC', '95% CI',
#                                                            'Sensitivity', 'Specificity', 'PPV', 'NPV', 'Precision', 'Recall', 'F1',
#                                                            'Threshold', 'Cohort'])
#         tauc = float(metric[(metric['model_name'] == sel_model) & (metric['Cohort'] == 'test')]['AUC'])
#         if tauc > maxi or tauc > 0.88:
#             maxi = tauc
#             print('-----------> ', maxi, p, md)
#         display(metric)

In [ ]:
import joblib
import os
from onekey_algo.custom.components.comp1 import plot_feature_importance, smote_resample
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

targets = []
os.makedirs('models', exist_ok=True)
for l in labels:
    new_models = okcomp.comp1.create_clf_model(model_names)
#     new_models['SVM'] = SVC(probability=True, kernel='rbf', max_iter=10)
    new_models['LR'] = LogisticRegression(penalty='l2', max_iter=100)
#     new_models['LightGBM'] = LGBMClassifier(n_estimators=5, max_depth=2, random_state=0)
    new_models['RandomForest'] = RandomForestClassifier(n_estimators=70, max_depth=3,
                                                        min_samples_split=2, random_state=0)
    new_models['ExtraTrees'] = ExtraTreesClassifier(n_estimators=5, max_depth=4, min_samples_split=2, random_state=0)
#     new_models['XGBoost'] = XGBClassifier(n_estimators=5, objective='binary:logistic', max_depth=1, min_child_weight=.2,
#                                               use_label_encoder=False, eval_metric='error')
    model_names = list(new_models.keys())
    new_models = list(new_models.values())
    for mn, m in zip(model_names, new_models):        
        X_train_sel, y_train_sel = X_data, y_data
#         X_train_sel, y_train_sel = smote_resample(X_train_sel, y_train_sel)
        m.fit(X_train_sel, y_train_sel[l])
        # 保存训练的模型
#         joblib.dump(m, f'models/{task_type}_{mn}_{l}.pkl') 
        # 输出模型特征重要性，只针对高级树模型有用
#         plot_feature_importance(m, selected_features[0], save_dir='img', prefix=f"{task_type}_")
    targets.append(new_models)

## 七、预测结果

* predictions，二维数据，每个label对应的每个模型的预测结果。
* pred_scores，二维数据，每个label对应的每个模型的预测概率值。

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from onekey_algo.custom.components.delong import calc_95_CI
from onekey_algo.custom.components.metrics import analysis_pred_binary

metric = []
pred_sel_idx = []
X_train_sel, y_train_sel = X_data, y_data
predictions = [[(model.predict(X_train_sel), 
                 [(model.predict(X_val_sel), y_val_sel) for X_val_sel, y_val_sel, _ in val_datasets.values()])  
                for model in target] for label, target in zip(labels, targets)]
pred_scores = [[(model.predict_proba(X_train_sel), 
                 [(model.predict_proba(X_val_sel), y_val_sel) for X_val_sel, y_val_sel, _ in val_datasets.values()]) 
                for model in target] for label, target in zip(labels, targets)]

# predictions[0].append(dl_preds)
# pred_scores[0].append(dl_pred_scores)
# model_names.extend(['WG', 'WOG'])
for label, prediction, scores in zip(labels, predictions, pred_scores):
    for mname, (train_pred, val_preds), (train_score, val_scores) in zip(model_names, prediction, scores):
        # 计算训练集指数
        acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y_train_sel[label], 
                                                                                              train_score[:, 1], use_youden=True)
        ci = f"{ci[0]:.3f} - {ci[1]:.3f}"
        metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, f"train"))
        for subset, (val_score, y_val_sel) in zip(val_datasets.keys(), val_scores):
            acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres = analysis_pred_binary(y_val_sel[label], 
                                                                                                  val_score[:, 1], use_youden=True)
            ci = f"{ci[0]:.3f} - {ci[1]:.3f}"
            metric.append((mname, acc, auc, ci, tpr, tnr, ppv, npv, precision, recall, f1, thres, subset))
metric = pd.DataFrame(metric, index=None, columns=['model_name', 'Accuracy', 'AUC', '95% CI',
                                                   'Sensitivity', 'Specificity', 'PPV', 'NPV', 'Precision', 'Recall', 'F1',
                                                   'Threshold', 'Cohort'])
metric

### 绘制曲线

绘制的不同模型的准确率柱状图和折线图曲线。

In [ ]:
import seaborn as sns

plt.figure(figsize=(15, 10))
plt.subplot(211)
sns.barplot(x='model_name', y='AUC', data=metric, hue='Cohort', hue_order=['train'] + subsets)
plt.legend(loc = 'lower right')
plt.ylim(0.5,)
plt.subplot(212)
sns.barplot(x='Cohort', y='AUC', data=metric, hue='model_name')
plt.legend(loc = 'lower right')
plt.ylim(0.5,)
plt.savefig(f'img/{task_type}_model_acc.svg', bbox_inches = 'tight')

## 绘制ROC曲线
确定最好的模型，并且绘制曲线。

```python
def draw_roc(y_test, y_score, title='ROC', labels=None):
```

`sel_model = ['SVM', 'KNN']`参数为想要绘制的模型对应的参数。

In [ ]:
sel_model = model_names

for sm in sel_model:
    if sm in model_names:
        sel_model_idx = model_names.index(sm)
    
        # Plot all ROC curves
#         plt.figure(figsize=(10, 10))
        for pred_score, label in zip(pred_scores, labels):
            ys = [np.array(y_train_sel[label])] + [np.array(y_val_sel[label]) for _, y_val_sel, _ in val_datasets.values()]
            ps = [pred_score[sel_model_idx][0]] + [p_[0] for p_ in pred_score[sel_model_idx][1]]
            okcomp.comp1.draw_roc(ys, ps, 
                                  labels=['Train'] + list(val_datasets.keys()), title=f"Model: {sm}")
            plt.savefig(f'img/{task_type}_model_{sm}_roc.svg', bbox_inches = 'tight')
            plt.show()

### 汇总所有模型

In [ ]:
sel_model = model_names

for pred_score, label in zip(pred_scores, labels):
    pred_val_scores = []
    for sm in sel_model:
        if sm in model_names:
            sel_model_idx = model_names.index(sm)
            pred_val_scores.append((pred_score[sel_model_idx][0], y_train_sel))
    p_, l_ = zip(*pred_val_scores)
    okcomp.comp1.draw_roc(l_, p_, labels=sel_model, title=f"Cohort train AUC")
    plt.savefig(f'img/{task_type}_model_train_roc.svg', bbox_inches = 'tight')
    plt.show()
    for sel_subset_idx, subset in enumerate(val_datasets.keys()):
        pred_val_scores = []
        for sm in sel_model:
            if sm in model_names:
                sel_model_idx = model_names.index(sm)
                pred_val_scores.append(pred_score[sel_model_idx][1][sel_subset_idx])
        p_, l_ = zip(*pred_val_scores)
        okcomp.comp1.draw_roc(l_, p_, labels=sel_model, title=f"Cohort {subset} AUC")
        plt.savefig(f'img/{task_type}_model_{subset}_roc.svg', bbox_inches = 'tight')
        plt.show()

In [ ]:
from onekey_algo.custom.components.comp1 import plot_DCA, draw_calibration
# 设置绘制参数
sel_model = model_names
for sm in sel_model:
    if sm in model_names:
        sel_model_idx = model_names.index(sm)
        for idx, label in enumerate(labels):
            for sel_subset_idx, subset in enumerate(val_datasets.keys()):
                p_ = pred_scores[idx][sel_model_idx][0]
                plot_DCA([p_[:, 0]], y_train_sel[label], title=f'Model for DCA', labels=sm, remap=True, y_min=-0.15, )
                plt.savefig(f'img/{task_type}_model_train_{sm}_dca.svg', bbox_inches = 'tight')
                plt.show()
                draw_calibration(pred_scores=[p_[:, 0]], n_bins=5, remap=True,
                                 y_test=[y_train_sel[label]], model_names=[sm])
                plt.savefig(f'img/{task_type}_model_train_{sm}_cali.svg', bbox_inches = 'tight')
                plt.show()
            for sel_subset_idx, subset in enumerate(val_datasets.keys()):
                p_, l_ = pred_scores[idx][sel_model_idx][1][sel_subset_idx]
                plot_DCA([p_[:, 0]], l_[label], title=f'Model for DCA', labels=sm, remap=True, y_min=-0.15, )
                plt.savefig(f'img/{task_type}_model_{subset}_{sm}_dca.svg', bbox_inches = 'tight')
                plt.show()
                draw_calibration(pred_scores=[p_[:, 0]], n_bins=5, remap=True,
                                 y_test=[l_[label]], model_names=[sm])
                plt.savefig(f'img/{task_type}_model_{subset}_{sm}_cali.svg', bbox_inches = 'tight')
                plt.show()

## 绘制混淆矩阵

绘制混淆矩阵，[混淆矩阵解释](https://baike.baidu.com/item/%E6%B7%B7%E6%B7%86%E7%9F%A9%E9%98%B5/10087822?fr=aladdin)
`sel_model = ['SVM', 'KNN']`参数为想要绘制的模型对应的参数。

如果需要修改标签到名称的映射，修改`class_mapping={1:'1', 0:'0'}`

In [ ]:
# 设置绘制参数
sel_model = model_names

label_mapping = get_param_in_cwd('label_mapping')
for sm in sel_model:
    if sm in model_names:
        sel_model_idx = model_names.index(sm)
        for idx, label in enumerate(labels):
            for sel_subset_idx, subset in enumerate(val_datasets.keys()):
                p_ = predictions[idx][sel_model_idx][0]
                cm = okcomp.comp1.calc_confusion_matrix(p_, y_train_sel[label], 
                                                        class_mapping=label_mapping, num_classes=2)
                plt.figure(figsize=(5, 4))
                plt.title(f'Model:{sm}')
                okcomp.comp1.draw_matrix(cm, norm=False, annot=True, cmap='Blues', fmt='.0f')
                plt.savefig(f'img/{task_type}_model_train_{sm}_cm.svg', bbox_inches = 'tight')
            for sel_subset_idx, subset in enumerate(val_datasets.keys()):
                p_, l_ = predictions[idx][sel_model_idx][1][sel_subset_idx]
                cm = okcomp.comp1.calc_confusion_matrix(p_, l_[label], 
                                                        class_mapping=label_mapping, num_classes=2)
                plt.figure(figsize=(5, 4))
                plt.title(f'Model:{sm}')
                okcomp.comp1.draw_matrix(cm, norm=False, annot=True, cmap='Blues', fmt='.0f')
                plt.savefig(f'img/{task_type}_model_{subset}_{sm}_cm.svg', bbox_inches = 'tight')

In [ ]:
import os
import numpy as np

os.makedirs('results', exist_ok=True)
sel_model = sel_model

all_log = []
for idx, label in enumerate(labels):
    for sm in sel_model:
        if sm in model_names:
            sel_model_idx = model_names.index(sm)
            target = targets[idx][sel_model_idx]
            # 预测训练集和测试集数据。
            train_indexes = np.reshape(np.array(train_ids), (-1, 1)).astype(str)
            # 保存预测的训练集和测试集结果
            y_train_pred_scores = target.predict_proba(X_data)
            columns = ['ID'] + [f"{label}-{i}"for i in range(y_train_pred_scores.shape[1])]
            result_train = pd.DataFrame(np.concatenate([train_indexes, y_train_pred_scores], axis=1), columns=columns)
            result_train.to_csv(f'results/{task_type}_{sm}_train.csv', index=False)
            result_train['model'] = sm
            result_train['pred_score'] = list(map(lambda x: max(x), 
                                                  np.array(result_train[['label-0', 'label-1']])))
            result_train['pred_label'] = list(map(lambda x: 0 if x[0] > x[1] else 1, 
                                                  np.array(result_train[['label-0', 'label-1']])))
            all_log.append(result_train)
            for subset, (X_val_sel, y_val_sel, val_ids) in val_datasets.items():
                val_indexes = np.reshape(np.array(val_ids), (-1, 1)).astype(str)
                y_val_pred_scores = target.predict_proba(X_val_sel)
                result_val = pd.DataFrame(np.concatenate([val_indexes, y_val_pred_scores], axis=1), columns=columns)
                result_val.to_csv(f'results/{task_type}_{sm}_{subset}.csv', index=False)
                result_val['model'] = sm
                result_val['pred_score'] = list(map(lambda x: max(x), np.array(result_val[['label-0', 'label-1']])))
                result_val['pred_label'] = list(map(lambda x: 0 if x[0] > x[1] else 1,  
                                                    np.array(result_val[['label-0', 'label-1']])))
                all_log.append(result_val)
all_log = pd.concat(all_log, axis=0)